In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import h5py
import os
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
# check if using gpu properly
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [9]:
# data loading functions

def load_cifar10():
    # load dataset
    (trainX, trainy), (testX, testy) = cifar10.load_data()
    
    trainY, testY = to_categorical(trainy), to_categorical(testy)
    # summarize data 
    print('Training Set: X: {}, y: {}'.format(trainX.shape, trainY.shape))
    print('Testing Set: X: {}, y: {}'.format(testX.shape, testy.shape))
    return trainX, trainY, testX, testY

def center(trainX, testX):
    '''
    Normalize images to have values between 0 and 1
    '''
    trainX_norm, testX_norm = trainX.astype('float32') / 255.0, testX.astype('float32') / 255.0
    return trainX_norm, testX_norm

def normalize(trainX, testX):
    mean = np.mean(trainX,axis=(0,1,2,3))
    std = np.std(testX,axis=(0,1,2,3))
    trainX_norm = (trainX.astype('float32') - mean) / (std+1e-9)
    testX_norm = (testX.astype('float32') - mean) / (std+1e-9)
    return trainX_norm, testX_norm


def load_and_center():
    trainX, trainY, testX, testY = load_cifar10()
    trainX_norm, testX_norm = center(trainX, testX)
    
    return trainX_norm, trainY, testX_norm, testY

def load_and_normalize():
    trainX, trainY, testX, testY = load_cifar10()
    trainX_norm, testX_norm = normalize(trainX, testX)
    
    return trainX_norm, trainY, testX_norm, testY
# print first 4 image
# for i in range(4):
#     plt.subplot(221 + i)
#     plt.imshow(trainX[i])

# plt.show()

In [10]:
def build_model_base1():
    model = Sequential()
    model.add(Input(shape=(32, 32, 3))) 
    f
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    opt = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

def build_model_base2():
    model = Sequential()
    model.add(Input(shape=(32, 32, 3))) 
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    opt = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

def build_model_base3():
    model = Sequential()
    model.add(Input(shape=(32, 32, 3))) 
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    opt = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [16]:
# batch_norm, l2 regularization, dropout
def build_model_VGG3_bn():
    model = Sequential()
    model.add(Input(shape=(32, 32, 3))) 
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    opt = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

def build_model_VGG3_l2():
    model = Sequential()
    model.add(Input(shape=(32, 32, 3))) 
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    opt = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

def build_model_VGG3_dropout():
    model = Sequential()
    model.add(Input(shape=(32, 32, 3))) 
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    opt = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [ ]:
def build_model_VGG3_final():
    model = Sequential()
    model.add(Input(shape=(32, 32, 3))) 
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), strides=1, padding='same', activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    opt = Adam(learning_rate=0.0003)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

In [11]:
def plot_history(history, model_name):
    '''
    Create 2 subplots:
    1. epoch v.s. training/testing loss 
    2. epoch v.s. training/testing accuracy
    '''
    fig, ax = plt.subplots(2, 2)
    plt.subplot(211)
    plt.plot(history.history['loss'], color='blue', label='taining')
    plt.plot(history.history['val_loss'], color='orange', label='testing')
    plt.xlabel('Epoch')
    plt.ylabel('Cross Entropy Loss')
    plt.title('Cross Entropy Loss - ' + model_name)
    plt.legend()
    plt.subplot(212)
    plt.plot(history.history['accuracy'], color='blue', label='taining')
    plt.plot(history.history['val_accuracy'], color='orange', label='testing')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy - ' + model_name)
    plt.legend()

    fig.tight_layout()
    plt.savefig('models/' + model_name + '/' + model_name + '_plot.png', dpi=300)
    plt.show()
    plt.close()

In [20]:
def train_and_validate(model, trainX_norm, trainY, testX_norm, testY):
    
    model_name = model.__name__[12:]
    # define model
    model = model()
    # fit
    print('Training ' + model_name + ' model')
    t = time.time()
    history = model.fit(trainX_norm, trainY, epochs=75, batch_size=64, validation_data=(testX_norm, testY), verbose=0)
    print('Training takes {} minutes'.format(round((time.time() - t) / 60, 3)))
    # evaluate
    loss, acc = model.evaluate(testX_norm, testY, verbose=0)
    
    print('> Testing accuracy - {}: {}%'.format(model_name, round(acc, 4) * 100))
    
    print('Saving model...')
    os.makedirs('models/'+model_name) 
    model.save('models/'+ model_name +'/' + model_name + '.h5')
    print('Model Saved')
    
    plot_history(history, model_name)
    return model

def train_and_validate_augmented(model, trainX_norm, trainY, testX_norm, testY):
    
    model_name = 'augmented_'+model.__name__[12:]
    # data augmentation
    datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
    
    # define model
    model = model()
    # fit
    print('Training ' + model_name + ' model')
    steps = trainX_norm.shape[0] // 64
    t = time.time()
    history = model.fit(datagen.flow(trainX_norm, trainY, batch_size=64), steps_per_epoch=steps, epochs=75, validation_data=(testX_norm, testY), verbose=1)
    print('Training takes {} minutes'.format(round((time.time() - t) / 60, 3)))
    # evaluate
    loss, acc = model.evaluate(testX_norm, testY, verbose=0)
    
    print('> Testing accuracy - {}: {}%'.format(model_name, round(acc, 4) * 100))
    
    print('Saving model...')
    os.makedirs('models/'+model_name)  
    model.save('models/'+ model_name +'/' + model_name + '.h5')
    print('Model Saved')
    
    plot_history(history, model_name)
    return model